In [94]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [178]:
def preprocess(imagefile_path):
   byte_img = tf.io.read_file(imagefile_path)
   img = tf.io.decode_jpeg(byte_img, channels=3)
   return img

In [192]:
images = tf.data.Dataset.list_files(os.path.join('data', '*.png'), shuffle=False)
images = images.map(preprocess)

In [ ]:
#image1 = images.as_numpy_iterator().next()
#plt.imshow(preprocess(os.path.join('data', '6a650b96-1e34-11ed-a7d9-201e887f9b25.png')))
#plt.imshow(image1)

In [219]:
def preprocess_keypress(file_path):
   context = tf.io.read_file(file_path)
   lines = tf.strings.split(context, '\r\n')
   return lines[0]

keypresses = tf.data.Dataset.list_files(os.path.join('data', '*.txt'), shuffle=False)
keypresses = keypresses.map(lambda x: tf.py_function(preprocess_keypress, [x], tf.int16))


# def warp_array(x1,x2):
#    return tf.expand_dims([x1,x2], axis=0)
# keypresses = keypresses.map(warp_array)



TypeError: 'MapDataset' object is not subscriptable

In [ ]:
# for x in keypresses.as_numpy_iterator():
#   print(x)

In [207]:
dataset = tf.data.Dataset.zip((images, keypresses))
dataset_len = len(dataset)
print(dataset_len)

143


In [214]:
batch_size = 3
if batch_size > dataset_len:
   batch_size = dataset_len
prefetch_len = dataset_len // batch_size

dataset = dataset.cache()
dataset = dataset.shuffle(dataset_len)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(prefetch_len)

In [184]:
def unpack_y(x, y):
   # 消除 y 的dim
   return x, tf.squeeze(y, axis=1)
dataset = dataset.map(unpack_y)

In [216]:
x, y = dataset.as_numpy_iterator().next()
len(y)

StopIteration: 

In [105]:
# plt.imshow(x[0])

In [106]:
# width, height, channels = images.as_numpy_iterator().next().shape
# print(width, height, channels)


Do Deep Learning Model

In [107]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization


In [212]:
from statistics import mode

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

model = Sequential()
# images.as_numpy_iterator().next().shape 得到 (800,)
model.add(Conv2D(64, 5, strides=(3,3), activation='relu', input_shape=(width, height, channels)))
model.add(Conv2D(128, 4, strides=(3,3), activation='relu'))
model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# keypresses.as_numpy_iterator().next()[0].shape 得到 (4,)

# 判斷 四 個按鈕
# model.add(Dense(18*31, activation='softmax'))

# softmax: 輸出各種結果的概率總和為1
# 預測 4 個數字
model.add(Dense(1, activation="relu", name="layer_predict_1_numbers"))
model.summary()


# 假如出現, 表示 model 耗用太多 memory
# OOM when allocating tensor with shape[60191872,128] and type float 


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 276, 146, 64)      4864      
                                                                 
 conv2d_33 (Conv2D)          (None, 91, 48, 128)       131200    
                                                                 
 flatten_16 (Flatten)        (None, 559104)            0         
                                                                 
 layer_predict_1_numbers (De  (None, 1)                559105    
 nse)                                                            
                                                                 
Total params: 695,169
Trainable params: 695,169
Non-trainable params: 0
_________________________________________________________________


In [209]:
# model.compile('adam', tf.losses.CategoricalCrossentropy())

# 判斷兩個數字用
model.compile(optimizer='adam', loss='mse')
#model.compile(optimizer='sgd', loss='mae')

In [210]:
# validation loss 三個執行週期沒改善就停止訓練
# tensorboard --logdir=.\logs
checkpoint_filepath = '.\\checkpoints\\checkpoint'
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='loss'),
    tf.keras.callbacks.TensorBoard(log_dir='.\\logs'),
    # monitor = val_acc loss val_loss
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='loss', verbose=1, save_freq='epoch', save_best_only=True, save_weights_only=True),
]

In [211]:

# 載入最近的檢查點的權重
#model.load_weights(checkpoint_filepath)
#model.fit(x, y, epochs=10)
model.fit(dataset, epochs=10, callbacks=my_callbacks)

Epoch 1/10


InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  ValueError: Tensor conversion requested dtype int16 for Tensor with dtype string: <tf.Tensor: shape=(), dtype=string, numpy=b'581'>
Traceback (most recent call last):

  File "D:\Users\flash\miniconda3\envs\tf2_jupyter\lib\site-packages\tensorflow\python\ops\script_ops.py", line 268, in __call__
    return func(device, token, args)

  File "D:\Users\flash\miniconda3\envs\tf2_jupyter\lib\site-packages\tensorflow\python\ops\script_ops.py", line 146, in __call__
    outputs = self._call(device, args)

  File "D:\Users\flash\miniconda3\envs\tf2_jupyter\lib\site-packages\tensorflow\python\ops\script_ops.py", line 171, in _call
    self._convert(ret, dtype=self._out_dtypes[0]), device_name)

  File "D:\Users\flash\miniconda3\envs\tf2_jupyter\lib\site-packages\tensorflow\python\ops\script_ops.py", line 130, in _convert
    return ops.convert_to_tensor(value, dtype=dtype)

  File "D:\Users\flash\minic